## Rivi Internship Assignment Task
<b>Submitted by : Yashi Kesarwani, 4th year CSE Undergraduate at IIIT Kalyani<b>

### Genetic Algorithm in brief:
1-Generating some random routes (Create Population)<br>
2-Finding the best routes among them (Determine fitness)<br>
3-Breeding some new random routes (Select matting pool and breed)<br>
4-Changing some random routes seperately (Mutate)<br>
5-Repeat steps 2-4<br> 

In [1]:
import math
import random
import matplotlib as plt

In [18]:
class Stop:
    x = None
    y = None
    def __init__(self, x = None, y = None):
        self.x = None
        self.x = None
        if x:
            self.x = x
        else:
            self.x = int(random.random() * 200)    
        if y:
            self.y = y
        else:
            self.y = int(random.random() * 200)
        
    def getx(self):
        return self.x
        
    def gety(self):
        return self.x
        
    def lengthofstop(self, stop):
        xlength = abs(self.getx() - stop.getx())
        ylength = abs(self.gety() - stop.gety())
        length = math.sqrt( (xlength*xlength) + (ylength*ylength) )
        return length
        
    def __repr__(self):
        return  str( self.getx()) + "," + str(self.gety())

In [19]:
class Salesman:
    destinationstops = []
    
    def addstop(self, stop):
        self.destinationstops.append(stop)
    
    def getstop(self, index):
        return self.destinationstops[index]
    
    def totalstops(self):
        return len(self.destinationstops)

In [20]:
class Travel:
    salesman = None
    travel = []
    fitness = 0.0
    length = 0
    
    def __init__(self, salesman, travel = None):
        self.salesman = salesman
        self.travel = []
        self.fitness = 0.0
        self.distance = 0
        if travel:
            self.travel = travel
        else:
            for i in range(0, self.salesman.totalstops()):
                self.travel.append(None)
            
    def lentravel(self):
        return len(self.travel)

    def getitem(self, index):
        return self.travel[index]
    
    def setitem(self, key, value):
        self.travel[key] = value
    
    def __repr__(self):
        genes = "|"
        for i in range(0, self.travelsize()):
            genes += str(self.getstop(i)) + " | "
        return genes
    
    def createRoute(self):
        for stopindex in range(0, self.salesman.totalstops()):
            self.setstop(stopindex, self.salesman.getstop(stopindex))
        random.shuffle(self.travel)
            
            
    def getstop(self, travelposition):
        return self.travel[travelposition]
    
    def setstop(self, travelposition, city):
        self.travel[travelposition] = city
        self.fitness = 0.0
        self.distance = 0
    
    def getfitness(self):
        if self.fitness == 0:
            self.fitness = 1/float(self.getlength())
        return self.fitness
    
    def getlength(self):
        if self.distance == 0:
            travellength = 0
            for stopindex in range(0, self.travelsize()):
                fromstop = self.getstop(stopindex)
                destinationstop = None
                if stopindex + 1 < self.travelsize():
                    destinationstop = self.getstop(stopindex+1)
                else:
                    destinationstop = self.getstop(0)
                travellength += fromstop.lengthofstop(destinationstop)
            self.length = travellength
        return self.length

    def travelsize(self):
        return len(self.travel)
    
    def isstop(self, stop):
        return stop in self.travel

In [21]:
class Population:
    travels = []
    
    def __init__(self, salesman , populationsize, initialize):

        self.travels = []
        for i in range (0, populationsize):
            self.travels.append(None)

        if initialize:
            for i in range(0, populationsize):
                newtravel = Travel(salesman)
                newtravel.createRoute()
                self.savetravel(i, newtravel)

    def setitem(self, key, value):
        self.travels[key] = value

    def getitem(self, index):
        return self.travels[index]

    def savetravel(self, index, travel):
        self.travels[index] = travel

    def gettravel(self, index):
        return self.travels[index]
    
    def getfittest(self):
        fittest = self.travels[0]
        for i in range(0, self.populationsize()):
            if fittest.getfitness() <= self.gettravel(i).getfitness():
                fittest = self.gettravel(i)
        return fittest
    
    def populationsize(self):
        return len(self.travels)

In [22]:
class GeneticAlgorithm:
    salesman = None
    mutationrate = 0.020
    tournamentsize = 7
    elitism = True

    def __init__(self, salesman):
        self.salesman = salesman
        self.mutationrate = 0.020
        self.tournamentsize = 7
        self.elitism = True

    def breedPopulation(self, pop):
        newpopulation = Population(self.salesman, pop.populationsize(), False)
        elitismOffset = 0
        if self.elitism:
            newpopulation.savetravel(0, pop.getfittest())
            elitismOffset = 1 

        for i in range(elitismOffset, newpopulation.populationsize()):
            parent1 = self.tournamentselection(pop)
            parent2 = self.tournamentselection(pop)
            child = self.changeroutes(parent1, parent2)
            newpopulation.savetravel(i, child)

        for i in range(elitismOffset, newpopulation.populationsize()):
            self.mutate(newpopulation.gettravel(i))

        return newpopulation 

    def changeroutes(self, parent1, parent2):
        child = Travel(self.salesman)

        startpoint = int(random.random() * parent1.travelsize())
        endpoint = int(random.random() * parent1.travelsize())

        for i in range(0, child.travelsize()):
            if startpoint < endpoint and i > startpoint and i < endpoint:
                child.setstop(i, parent1.getstop(i))
            elif startpoint > endpoint:
                if not (i < startpoint and i > endpoint):
                    child.setstop(i, parent1.getstop(i))

        for i in range(0, parent2.travelsize()):
            if not child.isstop(parent2.getstop(i)):
                for j in range(0, child.travelsize()):
                    if child.getstop(j) == None:
                        child.setstop(j, parent2.getstop(i))
                        break

        return child

    def mutate(self, travel):
        for travelpoint1 in range(0, travel.travelsize()):
            if random.random() < self.mutationrate:
                travelpoint2 = int(travel.travelsize() * random.random())

                stop1 = travel.getstop(travelpoint1)
                stop2 = travel.getstop(travelpoint2)

                travel.setstop(travelpoint1, stop1)
                travel.setstop(travelpoint2, stop2)

    def tournamentselection(self, pop):
        tournament = Population(self.salesman, self.tournamentsize, False)
        for i in range(0, self.tournamentsize):
            randomId = int(random.random() * pop.populationsize())
            tournament.savetravel(i, pop.gettravel(randomId))
        fittest = tournament.getfittest()
        return fittest

In [23]:
if __name__ == '__main__':
    
    salesman = Salesman()

    stops = [0]* 25
    for i in range(0,25):
        stops[i] = Stop(x=int(random.random() * 200), y=int(random.random() * 200))
        salesman.addstop(stops[i])

    #Create Population\
    pop = Population(salesman, 70, True)
    print ("Starting cost in the form of whole length of the path to be covered: " + str(pop.getfittest().getlength()))

    #Evolve population for 70 generations
    ga = GeneticAlgorithm(salesman)
    pop = ga.breedPopulation(pop)
    for i in range(0, 90):
        pop = ga.breedPopulation(pop)

    #printing the final path or route which is best to be travelled
    print ("After applying genetic algorithm")
    print ("Final cost in the form of whole length to be covered: " + str(pop.getfittest().getlength()))
    print ("Path or Route:")
    print (pop.getfittest())

Starting cost in the form of whole length of the path to be covered: 1770.5953800911152
After applying genetic algorithm
Final cost in the form of whole length to be covered: 1111.5718600252528
Path or Route:
|67,67 | 27,27 | 112,112 | 96,96 | 149,149 | 182,182 | 188,188 | 198,198 | 181,181 | 172,172 | 133,133 | 123,123 | 110,110 | 82,82 | 60,60 | 62,62 | 20,20 | 54,54 | 176,176 | 170,170 | 171,171 | 117,117 | 66,66 | 65,65 | 20,20 | 


#### Thank you for giving me this opporunity!